# Michał Gromadzki

## Load data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
SEED = 1337
tf.random.set_seed(SEED)

2023-06-26 17:10:48.727923: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-26 17:10:51.716141: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df_test = pd.read_csv("data/cybersecurity_test.csv",sep="|").drop(["n1","n2","n3","n4","n5","n6","n7","n8","n9","n10","score"],axis=1)
df_test.head()

,alert_ids,client_code,categoryname,ip,ipcategory_name,ipcategory_scope,parent_category,grandparent_category,overallseverity,timestamp_dist,...,thrcnt_week,thrcnt_day,p6,p9,p5m,p5w,p5d,p8m,p8w,p8d
0,Slg,RLJ,Exploit,MW.YB.50.64,INTERNET,Internet,7,A,3,0,...,298,42,1,0,1,1,1,1,1,1
1,WKM,UZT,Exploit,IJ.NW.77.74,INTERNET,Internet,7,A,5,0,...,11,3,1,0,1,1,1,1,1,1
2,dkm,ZZW,Attack,YT.LB.36.21,INTERNET,Internet,7,A,3,0,...,3601,602,1,0,3,1,1,1,1,1
3,RIX,QXG,Attack,172.BW.LB.105,PRIV-172,Private network,1,A,3,0,...,12,4,1,0,3,1,1,2,1,1
4,qFU,PDU,Exploit,YT.LB.32.110,INTERNET,Internet,7,A,3,258273,...,131,20,1,0,1,1,1,1,1,1


In [3]:
df_train = pd.read_csv("data/cybersecurity_training.csv",sep="|").drop(["n1","n2","n3","n4","n5","n6","n7","n8","n9","n10","score"],axis=1)
df_train.head()

,alert_ids,client_code,notified,categoryname,ip,ipcategory_name,ipcategory_scope,parent_category,grandparent_category,overallseverity,...,thrcnt_week,thrcnt_day,p6,p9,p5m,p5w,p5d,p8m,p8w,p8d
0,Nhq,DPM,0,Attack,YT.LB.32.21,INTERNET,Internet,7,A,3,...,4160,675,1,0,2,1,1,1,1,1
1,XZt,FIN,0,Exploit,192.SL.UK.94,PRIV-192,Private network,1,A,5,...,9,2,4,12,3,2,2,2,1,1
2,bBz,CHP,0,Attack,YT.LB.38.21,INTERNET,Internet,7,A,4,...,3788,628,1,0,2,2,1,2,2,1
3,ZNr,HPS,0,Attack,JX.NY.13.20,INTERNET,Internet,7,A,4,...,565,96,0,0,2,2,2,2,2,2
4,poV,OSC,0,Attack,YT.LB.32.21,INTERNET,Internet,7,A,4,...,2790,632,1,0,1,1,1,1,1,1


## Preprocess

In [4]:
res_train = np.load("data/result_train.npy",allow_pickle=True)
res_test = np.load("data/result_test.npy", allow_pickle=True)

In [5]:
res_train_df = pd.DataFrame(res_train,columns=["alert_ids","val1", "val2", "val3", "val4"])
res_test_df = pd.DataFrame(res_test,columns=["alert_ids","val1", "val2", "val3", "val4"])

In [6]:
#aggregate embeddigns
res_test_df = res_test_df.groupby("alert_ids").mean().astype("float32")
res_train_df = res_train_df.groupby("alert_ids").mean().astype("float32")

In [7]:
res_test_df.reset_index(inplace=True)
res_train_df.reset_index(inplace=True)

In [8]:
#add embedding to base data
df_test = df_test.merge(res_test_df, on="alert_ids")
df_train = df_train.merge(res_train_df, on="alert_ids")

In [9]:
labels_train = df_train["notified"].values
df_train = df_train.drop(["alert_ids","notified"],axis=1)
df_test = df_test.drop(["alert_ids"],axis=1)

In [10]:
#select column to tokenize
columns = df_train.columns
numerical_columns = df_train.describe().columns
transform_columns = [col for col in columns if col not in numerical_columns]
transform_columns

['client_code',
 'categoryname',
 'ip',
 'ipcategory_name',
 'ipcategory_scope',
 'grandparent_category',
 'weekday',
 'dstipcategory_dominate',
 'srcipcategory_dominate']

In [11]:
def preprocess_ip(text):
    text = tf.strings.split(text,sep=".")
    return text.to_tensor()

In [12]:
vectorizers = []
results_train = []
results_test = []
for col in transform_columns:
    std = preprocess_ip if col == "ip" else None
    vectorizer = tf.keras.layers.TextVectorization(standardize=std, split=None, name=f"{col}")
    vectorizer.adapt(df_train[f"{col}"].values)
    results_train.append(vectorizer(df_train[f"{col}"].values).numpy())
    results_test.append(vectorizer(df_test[f"{col}"].values).numpy())
    vectorizers.append(vectorizer)

2023-06-26 17:12:23.479619: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-26 17:12:23.799496: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-26 17:12:23.799618: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-26 17:12:23.806682: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-26 17:12:23.806816: I tensorflow/compile

In [13]:
#more columns to tokenize
transformcols2 = ["parent_category", "overallseverity", "start_hour", "start_minute", "start_second",
                    "isiptrusted", "untrustscore", "flowscore", "trustscore", "enforcementscore",
                    "dstportcategory_dominate", "srcportcategory_dominate", "p6", "p5m", "p5w", "p5d", "p8m", "p8w", "p8d"]

In [14]:
for col in transformcols2:
    vectorizer = tf.keras.layers.TextVectorization(standardize=None, split=None, name=f"{col}")
    x1, x2 = [str(item) for item in df_train[f"{col}"]], [str(item) for item in df_test[f"{col}"]]
    vectorizer.adapt(x1)
    results_train.append(vectorizer(x1).numpy())
    results_test.append(vectorizer(x2).numpy())
    vectorizers.append(vectorizer)

In [15]:
#rest of the columns - continous features
transform_columns.extend(transformcols2)
x_train = df_train[df_train.columns[~df_train.columns.isin(transform_columns)]].to_numpy().copy()
x_test = df_test[df_test.columns[~df_test.columns.isin(transform_columns)]].to_numpy().copy()

In [16]:
results_train = [np.reshape(np.array(item), (len(item), -1)) for item in results_train]
results_test = [np.reshape(np.array(item), (len(item), -1)) for item in results_test]

In [17]:
x_test.shape

(20000, 26)

In [18]:
x_train.shape

(39427, 26)

## Model

In [19]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='models/model_checkpoint',
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)

In [20]:
class emb_layer(tf.keras.layers.Layer):
    def __init__(self, units, n_embed, vectorizer):
        super(emb_layer, self).__init__()
        self.vectorizer = vectorizer
        self.n_embed = n_embed * 2 if self.vectorizer.name == "ip" else n_embed
        self.embedding = tf.keras.layers.Embedding(self.vectorizer.vocabulary_size(), self.n_embed, input_length=1)
        self.lstm1 = tf.keras.layers.LSTM(units, return_sequences=True)
        self.lstm2 = tf.keras.layers.LSTM(units)
        self.flat = tf.keras.layers.Flatten()
        self.mlp1 = tf.keras.layers.Dense(units, activation="relu")
        self.mlp2 = tf.keras.layers.Dense(units, activation="relu")

    def call(self, x):
        x = self.embedding(x)
        if self.vectorizer.name == "ip":
            x = self.lstm1(x)
            x = self.lstm2(x)
        else:
            x = self.flat(x)
        x = self.mlp1(x)
        x = self.mlp2(x)
        return x

In [21]:
class MyModel(tf.keras.Model):
    def __init__(self, units, n_embed, vectorizers):
        super(MyModel, self).__init__()
        self.emebd_layers = [emb_layer(units, n_embed, vec) for vec in vectorizers]
        self.conc = tf.keras.layers.Concatenate()
        self.mlp1 = tf.keras.layers.Dense(32, activation="relu")
        self.mlp2 = tf.keras.layers.Dense(64, activation="relu")
        self.main_mlp1 = tf.keras.layers.Dense(len(vectorizers)*units+64, activation="relu")
        self.main_mlp2 = tf.keras.layers.Dense(64, activation="relu")
        self.dropout = tf.keras.layers.Dropout(0)
        self.classifier = tf.keras.layers.Dense(1, activation="sigmoid")

    def call(self, inputs):
        main_inputs, emb_inputs = inputs

        xs = [embed_layer(emb_inputs[i]) for i,embed_layer in enumerate(self.emebd_layers)]
        x1 = self.mlp1(main_inputs)
        x1 = self.mlp2(x1)

        x = self.conc([x1,*xs])
        x = self.main_mlp1(x)
        x = self.main_mlp2(x)
        x = self.dropout(x)
        x = self.classifier(x)
        return x

In [22]:
model = MyModel(32,8, vectorizers)

In [23]:
model.compile(optimizer="adam", loss=tf.keras.losses.BinaryCrossentropy(), metrics=["accuracy", tf.keras.metrics.AUC()])

In [24]:
model.fit([x_train, results_train], labels_train, epochs=20, batch_size = 128, validation_split=0.2, callbacks=[checkpoint_callback])

Epoch 1/20


2023-06-26 17:16:53.630546: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-26 17:16:53.635241: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-26 17:16:53.638684: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

247/247 [==============================] - ETA: 0s - loss: 61.7108 - accuracy: 0.9070 - auc: 0.5748

2023-06-26 17:19:20.212046: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-26 17:19:20.216672: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-26 17:19:20.219652: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus


Epoch 1: val_loss improved from inf to 1.21324, saving model to models/model_checkpoint
247/247 [==============================] - 153s 287ms/step - loss: 61.7108 - accuracy: 0.9070 - auc: 0.5748 - val_loss: 1.2132 - val_accuracy: 0.8670 - val_auc: 0.7662
Epoch 2/20
247/247 [==============================] - ETA: 0s - loss: 0.7802 - accuracy: 0.9352 - auc: 0.7389
Epoch 2: val_loss improved from 1.21324 to 0.34604, saving model to models/model_checkpoint
247/247 [==============================] - 34s 138ms/step - loss: 0.7802 - accuracy: 0.9352 - auc: 0.7389 - val_loss: 0.3460 - val_accuracy: 0.9348 - val_auc: 0.7921
Epoch 3/20
247/247 [==============================] - ETA: 0s - loss: 0.2194 - accuracy: 0.9423 - auc: 0.8216
Epoch 3: val_loss improved from 0.34604 to 0.17423, saving model to models/model_checkpoint
247/247 [==============================] - 30s 121ms/step - loss: 0.2194 - accuracy: 0.9423 - auc: 0.8216 - val_loss: 0.1742 - val_accuracy: 0.9412 - val_auc: 0.8714
Epoch 4

In [25]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 emb_layer (emb_layer)       multiple                  3568      
                                                                 
 emb_layer_1 (emb_layer)     multiple                  1448      
                                                                 
 emb_layer_2 (emb_layer)     multiple                  29600     
                                                                 
 emb_layer_3 (emb_layer)     multiple                  1432      
                                                                 
 emb_layer_4 (emb_layer)     multiple                  1392      
                                                                 
 emb_layer_5 (emb_layer)     multiple                  1376      
                                                                 
 emb_layer_6 (emb_layer)     multiple                  141

In [26]:
model.load_weights("models/model_checkpoint")

In [27]:
preds = model.predict([x_test, results_test])
preds

2023-06-26 17:29:40.829661: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-26 17:29:40.833121: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-26 17:29:40.835950: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

625/625 [==============================] - 22s 32ms/step


array([[9.4982749e-03],
       [4.1262710e-01],
       [4.0728624e-05],
       ...,
       [2.0841895e-04],
       [1.0664350e-01],
       [1.1366428e-02]], dtype=float32)

In [ ]:
np.savetxt("solution.txt", preds)